In [10]:
!ls ../../papers_drug_discovery/'2 Computational Review Papers'/'2007 The role of multiscale computational approaches.pdf'

'../../papers_drug_discovery/2 Computational Review Papers/2007 The role of multiscale computational approaches.pdf'


In [235]:
!ls ../../papers_HPC/2407.04716v1.pdf

../../papers_HPC/2407.04716v1.pdf


In [11]:
! pip install pymupdf


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import fitz

In [13]:
from PIL import Image

In [442]:
import pymupdf

pdf = pymupdf.open("../../papers_drug_discovery/2 Computational Review Papers/2007 The role of multiscale computational approaches.pdf")
#pdf = pymupdf.open("../../papers_HPC/2407.04716v1.pdf")
page = pdf[1]

print("mediabox", page.mediabox)
print("cropbox", page.cropbox)

mediabox Rect(0.0, 0.0, 612.0, 792.0)
cropbox Rect(0.0, 0.0, 612.0, 792.0)


## Page Dict

In [443]:

doc_order = []

# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.get_text("dict")["blocks"]
for b in blocks:  # iterate through the text blocks
    #print(b)
    if "ext" in b:
        if b["ext"] == "jpeg" or b["ext"] == "png":
            doc_order.append(["image", b["bbox"][0], b["bbox"][1], b["bbox"][2], b["bbox"][3]])
        #else:
            #print(b)
    elif "lines" in b:
        #print(b["bbox"])
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                doc_order.append(["text", s["bbox"][0], s["bbox"][1], s["bbox"][2], s["bbox"][3]])
    #else:
        #print(b)

#for d in doc_order:
#    print(d)
            

In [444]:
# What range of coordinates does text content take up?
minx = doc_order[0][1]
miny = doc_order[0][2]
maxx = doc_order[0][1]
maxy = doc_order[0][2]

for d in doc_order:
    if d[1] < minx:
        minx = d[1]

    if d[1] > maxx:
        maxx = d[1]

    if d[2] < miny:
        miny = d[2]

    if d[1] > maxy:
        maxy = d[2]

    if d[3] < minx:
        minx = d[3]

    if d[3] > maxx:
        maxx = d[3]

    if d[4] < miny:
        miny = d[4]

    if d[4] > maxy:
        maxy = d[4]

print(minx, miny, maxx, maxy)

53.849998474121094 28.74383544921875 543.7523193359375 762.0636596679688


## Annotations

In [445]:
page_content = page

In [446]:
def two_lines_overlap(shape_items):
    abs(shape_items[0][1] - shape_items[1][1]) < 2

def two_lines_cross(shape_items):
    print(shape_items)
    print(shape_items[0])
    print(shape_items[0][1])
    
    l1y1 = shape_items[0][1][1]
    l1y2 = shape_items[0][2][1]
    if l1y1 > l1y2:
        l1maxy = l1y1
        l1miny = l1y2
    else:
        l1maxy = l1y2
        l1miny = l1y1

    l2y1 = shape_items[1][1][1]
    l2y2 = shape_items[1][2][1]
    if l1y1 > l1y2:
        l2maxy = l1y1
        l2miny = l1y2
    else:
        l2maxy = l1y2
        l2miny = l1y1

    if l1maxy > l2miny + 1:
        if l1miny < l2maxy - 1:
            print("cross")
            return True

    print(" no cross")
    return False
    

In [447]:
def remove_shape(shape_type, shape_rect, shape_items):
    if shape_type == "s":

        if len(shape_items) == 1:
            return True

        if len(shape_items) == 2:
            if two_lines_overlap(shape_items):
                return True
            elif two_lines_cross(shape_items):
                return False
            else: # they must be parallel
                return True

    if shape_type == "f":
        # if it is basically a line
        print("f", abs(shape_rect[1] - shape_rect[3]))
        if abs(shape_rect[1] - shape_rect[3]) < 2:
            return True
    

In [448]:
def insert_in_doc_order(shape):
    i = 0
    not_found = True
    while i < len(doc_order) and not_found:
        #print(i, doc_order[i][0],shape[0])
        if doc_order[i][2] > shape[2]: # comparing on y coordinate
            #print(i, "inserted")
            doc_order.insert(i, shape)
            not_found = False
        i = i + 1

In [449]:
from pymupdf import Rect, Point

newdoc = pymupdf.open()
page = newdoc.new_page()
nshape = page.new_shape()

i = 0
for shape in page_content.get_drawings():
    print(i)
    i = i + 1
    #if shape["type"] == "s":
#    print("shape", shape["type"])
#    print(shape["items"])
#    print(shape["rect"])
    if not remove_shape(shape["type"], shape["rect"], shape["items"]):
        insert_in_doc_order(["shape" + shape["type"],  shape["rect"][0],  shape["rect"][1],  shape["rect"][2],  shape["rect"][3]])
        for item in shape['items']:
            if item[0] == 're':
                nshape.draw_rect(item[1])
            elif item[0] == 'l':
                nshape.draw_line(item[1], item[2])
            elif item[0] == 'c':
                nshape.draw_bezier(item[1], item[2], item[3], item[4])

#        else:
#            print("------------unknown-------------", ["shape",  shape["type"], shape["rect"][1], shape["items"], shape["rect"]])
    
        #print(shape['type'], shape['fill_opacity'], shape['fill'],
        #      shape['type'], shape['stroke_opacity'], shape['color'], shape['width'], shape['lineCap'], 
        #      shape['lineJoin'], shape['closePath'], shape['dashes'], shape['layer'], 
        #      shape['seqno'], shape['fill'], shape['fill_opacity'], shape['even_odd'])
        #nshape.finish(width = shape['width'], fill=shape['fill'])
        #shape_details = {'fill': shape['fill']}
    nshape.finish(fill=shape['fill'])

nshape.commit(overlay=True)
newdoc.save("output.pdf")

for page in newdoc:
    pix = page.get_pixmap()
    pix.save("newdoc.png")

0
[('l', Point(53.849998474121094, 749.5499877929688), Point(541.1500244140625, 749.5499877929688)), ('l', Point(53.849998474121094, 39.95001220703125), Point(541.1500244140625, 39.95001220703125))]
('l', Point(53.849998474121094, 749.5499877929688), Point(541.1500244140625, 749.5499877929688))
Point(53.849998474121094, 749.5499877929688)
 no cross
1
f 42.5
2
f 14.0
3
f 40.6669921875
4
f 14.25
5
f 14.25
6
f 14.25
7
f 14.25
8
f 14.25
9
f 14.25
10
f 14.25
11
f 14.25
12
f 14.25
13
f 82.0


In [450]:
import pandas as pd

In [451]:
df = pd.DataFrame(doc_order, columns = ['type', 'x0', 'y0', 'x1', 'y1'])

In [452]:
from tabulate import tabulate

In [453]:
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

+-----+--------+----------+----------+----------+----------+
|     | type   |       x0 |       y0 |       x1 |       y1 |
|-----+--------+----------+----------+----------+----------|
|   0 | shapef | 162.305  | 521.469  | 240.555  | 563.969  |
|   1 | shapef | 409.474  | 556.301  | 491.474  | 638.301  |
|   2 | shapef | 273.805  | 594.636  | 382.139  | 635.303  |
|   3 | shapef |  79.805  | 598.719  | 153.805  | 612.719  |
|   4 | shapef |  90.805  | 688.969  | 115.055  | 703.219  |
|   5 | shapef | 125.055  | 689.969  | 154.305  | 704.219  |
|   6 | shapef | 192.805  | 692.469  | 222.055  | 706.719  |
|   7 | shapef | 295.305  | 692.969  | 325.555  | 707.219  |
|   8 | shapef | 230.055  | 693.219  | 260.305  | 707.469  |
|   9 | shapef | 332.805  | 693.719  | 363.055  | 707.969  |
|  10 | shapef | 435.555  | 693.719  | 465.805  | 707.969  |
|  11 | shapef | 475.805  | 693.969  | 506.055  | 708.219  |
|  12 | shapef | 396.555  | 694.219  | 426.805  | 708.469  |
|  13 | text   | 384.255